In [1]:
import numpy as np
from scipy import signal
from pydub import AudioSegment
from pydub.playback import play
from bokeh.io import show, output_notebook
from bokeh.plotting import figure 
import warnings

warnings.filterwarnings('ignore')
output_notebook()

Loading BokehJS ...

In [70]:
class AudioFP():
    ## Initializing AudioFP object properties
    def __init__(self, filename):
        self.songname = filename
        self.channels = []
        self.songdata = []
        self.framerate = []
        
    ## Read audio file using pydub and plot signal
    def read_audiofile(self):
        audiofile = AudioSegment.from_file(self.songname)
        self.songdata = np.frombuffer(audiofile._data, np.int16)
        for chn in range(audiofile.channels):
            self.channels.append(self.songdata[chn::audiofile.channels])
        self.framerate = audiofile.frame_rate
        p1 = figure(plot_width=800, plot_height=500, title='Audio Signal', 
                    x_axis_label='Time (s)', y_axis_label='Amplitude (arb. units)')
        time = np.linspace(0, len(self.channels[0])/self.framerate, len(self.channels[0]))
        p1.line(time[0::1000], self.channels[0][0::1000])
        show(p1)
        
    ## Generate and plot spectrogram of audio data
    def generate_spectogram(self):
        audiosignal = self.channels[0]
        fs = self.framerate
        window = 'hamming'
        nperseg = 4096
        overlap_ratio = 0.5
        noverlap = int(overlap_ratio * nperseg)
        f, t, sgram = signal.spectrogram(audiosignal, fs, window)
        sgram = 10 * np.log10(sgram)  # apply log transform since specgram() returns linear array
        sgram[sgram == -np.inf] = 0  # replace infs with zeros
        p2 = figure(plot_width=800, plot_height=500, title='Spectrogram',
                    x_axis_label='Time (s)', y_axis_label='Frequency (Hz)',
                    x_range=(0, max(t)), y_range=(0, max(f)))
        p2.image([sgram[::2, ::2]], x=0, y=0, dw=max(t), dh=max(f), palette='Spectral11')
        show(p2)

In [71]:
song1 = AudioFP("Choc--Eigenvalue-Subspace-Decomposition.mp3")

In [72]:
song1.read_audiofile()

In [73]:
song1.generate_spectogram()

ValueError: expected an element of Seq(Color), got 'Jet11'